# Домашнее задание - линейная регрессия

### Работа с признаками (8 баллов)

Скачайте датасет из материалов к уроку или по ссылке https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv 


Описание признаков:

* Agency — название страхового агентства
* Agency Type — тип страхового агентства
* Distribution Channel — канал продвижения страхового агентства
* Product Name — название страхового продукта
* Duration — длительность поездки (количество дней)
* Destination — направление поездки
* Net Sales — сумма продаж 
* Commission (in value) — комиссия страхового агентства
* Gender — пол застрахованного
* Age — возраст застрахованного

Ответ:
* Claim — потребовалась ли страховая выплата: «да» — 1, «нет» — 0

Обработайте пропущенные значения и примените написанные функции onehot_encode() и minmax_scale().

**Подсказка**: маску для категориальных признаков можно сделать фильтром cat_features_mask = (df.dtypes == "object").values

In [170]:
# < напишите код здесь >
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix

# Функция для one hot encoding
def onehot_encoding(x):
    values = np.unique(x)
    result = np.zeros((len(x), len(values)))
    for i, j in enumerate(x):
        # Находим позицию значения в уникальных значениях
        result[i, np.where(values == j)[0][0]] = 1
    return result.astype(int)

# Функция для min-max масштабирования
def minmax_scale(x):
    min_ = x.min(axis=0)
    max_ = x.max(axis=0)
    diff = max_ - min_
    diff[diff == 0] = 1  # избегаем деления на ноль
    return (x - min_) / diff

# Загрузка датасета
url = "https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv"
df = pd.read_csv(url)

# Разделяем признаки на числовые и категориальные
cat_features_mask = (df.dtypes == "object").values
cat_features = df.columns[cat_features_mask]
df_num = df[df.columns[~cat_features_mask]].copy()
df_cat = df[df.columns[cat_features_mask]].copy()

# Импутация пропусков в числовых данных (замена на среднее)
imputer = SimpleImputer(strategy="mean")
df_num = pd.DataFrame(imputer.fit_transform(df_num), columns=df_num.columns)

# Замена пропусков в категориальных данных на пустую строку
df_cat = df_cat.fillna("")

# Если колонка 'Duration' присутствует, удаляем выбросы одновременно для числовых и категориальных данных
if "Duration" in df_num.columns:
    Q1 = df_num["Duration"].quantile(0.25)
    Q3 = df_num["Duration"].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    mask = (df_num["Duration"] >= lower_bound) & (df_num["Duration"] <= upper_bound)
    before_rows = df_num.shape[0]
    df_num = df_num[mask]
    df_cat = df_cat[mask]  # применяем ту же маску к категориальным данным

# Применяем one hot encoding для категориальных признаков
df_cat_en = pd.DataFrame()
for col in cat_features:
    # Если в признаке слишком много уникальных значений, можно его пропустить (например, 'Duration')
    if col == "Duration" or df_cat[col].nunique() > 50:
        continue
    encoded = onehot_encoding(df_cat[col].values)
    encoded_df = pd.DataFrame(encoded,
                              columns=[f"{col}_{val}" for val in np.unique(df_cat[col])],
                              index=df_cat.index)
    df_cat_en = pd.concat([df_cat_en, encoded_df], axis=1)

# Масштабирование числовых признаков
df_ms = pd.DataFrame(minmax_scale(df_num), columns=df_num.columns, index=df_num.index)

# Объединение масштабированных числовых данных и обработанных категориальных признаков
df_new = pd.concat([df_ms, df_cat_en], axis=1)
df_new.head(25)

,Duration,Net Sales,Commision (in value),Age,Agency_ADM,Agency_ART,Agency_C2B,Agency_CBH,Agency_CCR,Agency_CSR,...,Product Name_Single Trip Travel Protect Silver,Product Name_Ticket Protector,Product Name_Travel Cruise Protect,Product Name_Travel Cruise Protect Family,Product Name_Value Plan,Claim_No,Claim_Yes,Gender_,Gender_F,Gender_M
2,0.553719,0.307836,0.161290,0.271186,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,0.512397,0.320149,0.129032,0.271186,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,0.669421,0.344776,0.064516,0.347458,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
5,0.561983,0.218905,0.229988,0.372881,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
6,0.404959,0.320149,0.129032,0.271186,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
7,0.537190,0.233955,0.354839,0.245763,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
8,0.487603,0.344776,0.064516,0.372881,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
10,0.289256,0.337065,0.049419,1.000000,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
11,0.024793,0.347015,0.034213,0.398305,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
12,0.454545,0.207711,0.268274,0.406780,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


Подробный анализ и подготовка датасета часто помогают улучшить качество модели. Ниже представлено несколько идей преобразований. Вы можете применить одно или несколько из этих преобразований (а можете не применять), чтобы помочь будущей модели. 

1. Посмотрите на количественные признаки. Возможно, в некоторых признаках есть выбросы - значения, которые сильно выбиваются. Такие значения полезно удалять. Советуем присмотреться к колонке Duration)

2. Можно заметить, что one hot encoding сильно раздувает количество столбцов. Радикальное решение - можно попробовать выбросить все категориальные признаки из датасета.

3. Если все-таки оставляете категориальные признаки, то подумайте, как уменьшить количество столбцов после one hot encoding. Признаки с большим количеством значений (Duration - 149! разных стран) можно удалить или попробовать сгруппировать некоторые значения.

4. Downsampling. Датасет достаточно большой, разница в классах огромная. Можно уменьшить число наблюдений с частым ответом.

In [173]:
# < напишите код здесь >
# Предложенные методы реализованы в предыдущем блоке кода, а также, например,
# в следующем блоке применен алгоритм уменьшения числа скоррелированных признаков,
# для того чтобы матрица была обратимой.

### Применение линейной регрессии (10 баллов)

Это задача классификации, но её можно решить с помощью линейной регрессии, если округлять предсказанный ответ до целого и выбирать ближайший по значению ответ из множества {0, 1}.

Вынесите признак 'Claim' в вектор ответов и разделите датасет на обучающую и тестовую выборку в соотношении 80 к 20. Зафиксируйте random_state.

**Подсказка:** быстро перевести Yes/No в 1/0 можно так - np.where(df['Claim'] == 'Yes', 1,0)

In [177]:
# Выделяем целевую переменную (Claim_Yes) и признаки (X)
X = df_new.drop(columns=["Claim_Yes", "Claim_No"], errors="ignore").values  # Преобразуем в NumPy
y = df_new["Claim_Yes"].values  # Целевая переменная тоже в NumPy

def ensure_full_rank(X, tol=1e-5):
    #Удаляет линейно зависимые признаки до тех пор, пока ранг матрицы не станет равен числу признаков.
    while np.linalg.matrix_rank(X) < X.shape[1]:
        U, S, Vt = np.linalg.svd(X, full_matrices=False)
        rank = np.sum(S > tol)

        if rank == X.shape[1]:
            break  # Если ранг уже равен числу признаков, выходим из цикла
        # Оставляем только первые `rank` независимых столбцов
        X = X[:, :rank]

    return X

# Применяем метод, пока не добьёмся полной ранговой матрицы
X = ensure_full_rank(X)

# Разделяем выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Проверяем финальный ранг
final_rank = np.linalg.matrix_rank(X_train)
num_features = X_train.shape[1]
print(f" Финальный ранг X_train: {final_rank}, Число признаков: {num_features}")

 Финальный ранг X_train: 19, Число признаков: 19


Найдите аналитическое решение для обучающей выборки: обычное и регуляризацией l2. 

In [180]:
# Добавляем столбец единиц для учёта intercept (свободного члена)
X_train_1 = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
X_test_1  = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

# Считаем матрицу (X^T X) и (X^T y)
XTX = X_train_1.T @ X_train_1
XTy = X_train_1.T @ y_train

# Аналитическое решение
w_ = np.linalg.inv(XTX) @ XTy

# Предсказания на тесте
y_pred = X_test_1 @ w_

# MSE
mse_ = mean_squared_error(y_test, y_pred)
# Для классификации берём порог 0.5
y_pred_class = (y_pred >= 0.5).astype(int)
acc_ = accuracy_score(y_test, y_pred_class)


print("Обычная лин. регрессия")
print(f"MSE: {mse_:.4f}")
print(f"Accuracy: {acc_:.4f}")

Обычная лин. регрессия
MSE: 0.0190
Accuracy: 0.9884


In [182]:
# Пересчитываем XTy (так как X_train_1 обновился)
XTy = X_train_1.T @ y_train  

# Определяем параметр регуляризации
lambda_ = 1.0  # Коэффициент регуляризации
n_features = X_train_1.shape[1]  # Число признаков (включая единичный столбец)

# Формируем (X^T X + λI) и решаем
XTX_ridge = X_train_1.T @ X_train_1 + lambda_ * np.eye(n_features)
w_ridge = np.linalg.inv(XTX_ridge) @ XTy

# Предсказания на тесте
y_pred_ridge = X_test_1 @ w_ridge

# Оцениваем MSE и Accuracy
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
y_pred_ridge_class = (y_pred_ridge >= 0.5).astype(int)
acc_ridge = accuracy_score(y_test, y_pred_ridge_class)

# Выводим результаты
print("Ridge-регрессия (L2)")
print(f"lambda = {lambda_}")
print(f"MSE: {mse_ridge:.4f}")
print(f"Accuracy: {acc_ridge:.4f}")


Ridge-регрессия (L2)
lambda = 1.0
MSE: 0.0111
Accuracy: 0.9884


Постройте модель LinearRegression, примените к тестовой выборке и посчитайте MSE (можно использовать библиотеку sklearn)

In [193]:
# обучите модель линейной регрессии LinearRegression на обучающей выборке, примените к тестовой
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Создаём и обучаем модель линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)  # Обучение на тренировочных данных

# Делаем предсказания на тестовой выборке
y_pred = model.predict(X_test)

In [197]:
# посчитайте MSE, предварительно округлив предсказанные ответы до целого
# Округляем предсказания до ближайшего целого (0 или 1)
y_pred_rounded = np.round(y_pred)

# Вычисляем MSE
mse_lr = mean_squared_error(y_test, y_pred_rounded)
acc_lr = accuracy_score(y_test, y_pred_rounded)
# Выводим результат
print(f"MSE на тестовой выборке: {mse_lr:.4f}")
print(f"MSE на тестовой выборке: {acc_lr:.4f}")

MSE на тестовой выборке: 0.0116
MSE на тестовой выборке: 0.9884


### Вывод (1 балла)

Напишите краткий вывод по заданию (достаточно пары предложений). Расскажите, какие способы предобработки данных вы выбрали и почему. Насколько хороша ваша модель?

In [203]:
print("Итоговое сравнение моделей")

print(f"Обычная лин. регрессия:     MSE = {mse_:.4f}, Accuracy = {acc_:.4f}")
print(f"Ridge-регрессия (L2):       MSE = {mse_ridge:.4f}, Accuracy = {acc_ridge:.4f}")
print(f"LinearRegression (sklearn):      MSE = {mse_lr:.4f}, Accuracy = {acc_lr:.4f}")


Итоговое сравнение моделей
Обычная лин. регрессия (OLS):     MSE = 0.0190, Accuracy = 0.9884
Ridge-регрессия (L2, λ²):       MSE = 0.0111, Accuracy = 0.9884
LinearRegression (sklearn):      MSE = 0.0116, Accuracy = 0.9884


In [205]:
"""
Вывод:
В данной задаче была использована линейная регрессия, Ridge-регрессия (L2) и LinearRegression из sklearn 
для решения задачи бинарной классификации. 

Предобработка данных включала:
- Масштабирование числовых признаков (Min-Max Scaling)
- One-hot encoding для категориальных данных
- Удаление линейно зависимых признаков (SVD-анализ)
- Устранение выбросов в числовых данных

Результаты:
- Ridge-регрессия (L2) показала наименьший MSE, что говорит о её лучшей устойчивости и меньшем переобучении.
  MSE = 0.0111, Accuracy = 0.9884 - модель очень хорошо справляется с задачей классификации
"""


'\nВывод:\nВ данной задаче была использована линейная регрессия, Ridge-регрессия (L2) и LinearRegression из sklearn \nдля решения задачи бинарной классификации. \n\nПредобработка данных включала:\n- Масштабирование числовых признаков (Min-Max Scaling)\n- One-hot encoding для категориальных данных\n- Удаление линейно зависимых признаков (SVD-анализ)\n- Устранение выбросов в числовых данных\n\nРезультаты:\n- Ridge-регрессия (L2) показала наименьший MSE, что говорит о её лучшей устойчивости и меньшем переобучении.\n  MSE = 0.0111, Accuracy = 0.9884 - модель очень хорошо справляется с задачей классификации\n'